In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame.from_csv('all_games.csv', index_col=None)
df.head()

,fran_id,pts,opp_fran,opp_pts,game_location,date
0,Rockets,120,Hawks,104,A,1976-10-21
1,Celtics,129,Pacers,122,A,1976-10-21
2,Clippers,133,Bucks,112,A,1976-10-21
3,Lakers,97,Knicks,102,A,1976-10-21
4,Bulls,95,Cavaliers,106,A,1976-10-22


In [3]:
a = df['pts']
b = df['opp_pts']
df['A Win'] = df['pts']>df['opp_pts']
df['A Win'] = df['A Win'].astype(int)
df[:20]

,fran_id,pts,opp_fran,opp_pts,game_location,date,A Win
0,Rockets,120,Hawks,104,A,1976-10-21,1
1,Celtics,129,Pacers,122,A,1976-10-21,1
2,Clippers,133,Bucks,112,A,1976-10-21,1
3,Lakers,97,Knicks,102,A,1976-10-21,0
4,Bulls,95,Cavaliers,106,A,1976-10-22,0
5,Nets,104,Warriors,103,A,1976-10-22,1
6,Nuggets,123,Pacers,110,A,1976-10-22,1
7,Pistons,99,Kings,96,A,1976-10-22,1
8,Suns,98,Jazz,111,A,1976-10-22,0
9,Spurs,121,Sixers,118,A,1976-10-22,1


In [4]:
teams = df['fran_id'].unique()
zeros = np.zeros(df.shape[0])
#print teams

for i in teams:
    #home indices and their points
    indices_home = df[(df['fran_id']==i)].index
    points_home = df[(df['fran_id']==i)].pts
    
    
    #away indices and their points
    indices_away = df[(df['opp_fran']==i)].index
    points_away = df[(df['opp_fran']==i)].pts
    
    #print points_home
    
    #print len(list(indices_home))
    
    # stack the two together and create a df for each team
    ind = (indices_home).tolist() + (indices_away).tolist()
    poi = points_home.tolist() + points_away.tolist()
    #print ind
    #print poi
    teamdf = pd.DataFrame()
    
    teamdf['index'] = ind
    teamdf['points'] = poi
    
    teamdf = teamdf.sort_values('index')
    teamdf = teamdf.reset_index(drop=True)
    teamdf['points'] = pd.rolling_mean(teamdf['points'], 5)
    teamdf.columns = ['index', i]
    teamdf.index = teamdf['index']
    teamdf = teamdf.drop(['index'], axis=1)
    #print teamdf[:20]
    df = df.join(teamdf)
    #print df[-20:]


#print indices

/Users/adammorrow1/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()


In [8]:
df = df.fillna(0)
def findFranAverage(row):
    fran = row['fran_id']
    return row[fran]
def findOppAverage(row):
    opp = row['opp_fran']
    return row[opp]
df['Fran_5_PTS'] = df.apply(findFranAverage,axis=1)
df['Opp_5_PTS'] = df.apply(findOppAverage,axis=1)

for i in teams:
    df = df.drop(i, axis=1)
#print df[teams].sum(axis=1)
#print df[:20]

In [9]:
df[-20:]

,fran_id,pts,opp_fran,opp_pts,game_location,date,A Win,Fran_5_PTS,Opp_5_PTS
47447,Warriors,85,Spurs,107,A,2017-03-11,0,103.4,96.2
47448,Bulls,80,Celtics,100,A,2017-03-12,0,96.4,97.2
47449,Knicks,112,Nets,120,A,2017-03-12,0,104.4,110.2
47450,Cavaliers,112,Rockets,117,A,2017-03-12,0,105.4,112.0
47451,Heat,98,Pacers,102,A,2017-03-12,0,97.2,93.2
47452,Sixers,118,Lakers,116,A,2017-03-12,1,114.8,114.2
47453,Trailblazers,110,Suns,101,A,2017-03-12,1,117.0,113.8
47454,Bulls,115,Hornets,109,A,2017-03-13,1,99.2,102.0
47455,Lakers,101,Nuggets,129,A,2017-03-13,0,111.4,104.8
47456,Bucks,93,Grizzlies,113,A,2017-03-13,0,95.6,108.8


In [11]:
df.to_pickle('contrived.pickle')